# Import eines Pickle-Files

In [1]:
import pandas as pd
import pickle
import os

rootdir = os.getcwd() + "/data/training/"

print(rootdir)

data = []
labels = []

for subdir, dirs, files in os.walk(rootdir):
    for fl in files:
        if fl.endswith('.pkl'):
            # string split for event labelling
            str_arr = fl.split('_')
            label   = str_arr[1]

            # read data and append it to variable
            infile = open((rootdir + fl),'rb')
            new_dict = pickle.load(infile, encoding='latin1')

            data.append(new_dict)
            labels.append(label)



/Users/maximilianrohleder-kirsch/aiproj/data/


# Read and combine data of some activities

In [83]:
import numpy as np 
from collections import defaultdict

keys = data[0].keys()

unique_keys = []
[unique_keys.append(k) for k in keys if k not in unique_keys]

bending = []
bending_dict = defaultdict(list)

# Collecting Bending data
for i in range(0, 9):
    for key in data[i].keys():
        x = data[i][key]
        y = key
        bending_dict[y].append(x)

concat_data = defaultdict(list)

for k in unique_keys:
    for i in bending_dict[k]:
        concat_data[k].append(i)

    concat_data[k] = np.vstack(concat_data[k])

# ToDo: Mean, Varianz, Minimum, Maximum, Differenz Minimum - Maximum



-8.9375
0.002130528966666667
